# Example:Allocating Balances

## Imports

In [2]:
########################################################################
## FOR NOTEBOOKS ONLY: ADD THE PROJECT ROOT TO THE PYTHON PATH
########################################################################

import os
import sys

sys.path.insert(
    0, os.path.abspath(os.path.join(os.getcwd(), '..'))
)

In [3]:
import datetime

from sinkingfund.allocation.sorted import SortedAllocator
from sinkingfund.utils.loader import load_envelopes_from_csv

## Load Bills

In [19]:
path = 'data/schwab_fund.csv'
interval = 14

envelopes = load_envelopes_from_csv(path=path, contrib_intervals=interval)

## Allocate Balance

In [6]:
# The start date of the contribution schedule. The current balance must
# match the balance at the start date. If there are unpaid bills in the
# account, you manually need to remove their amounts from the balance.
start_date = datetime.date(2025, 6, 1)
balance = 15026.41

### Cascading Allocation

In [12]:
# Initialize the allocator.
allocator = SortedAllocator(sort_key='cascade')

# Allocate the balance to the envelopes.
allocator.allocate(envelopes=envelopes, balance=balance, curr_date=start_date)

# Verify total allocated equals budget.
print(sum([e.allocated for e in envelopes if e.allocated is not None]) == balance)
print([(e.bill.bill_id, e.bill.amount_due, e.allocated, e.remaining) for e in envelopes])

True
[('car_insur_1', 774.76, None, None), ('suburu_reg', 191.0, 0.0, 191.0), ('honda_reg', 334.0, 0.0, 334.0), ('prop_tax_supl_2024', 11520.23, 11520.23, 0.0), ('car_insur', 774.76, 774.76, 0.0), ('prop_tax_1', 13161.15, 2731.42, 10429.73), ('prop_tax_2', 13161.15, 0.0, 13161.15), ('home_insur', 3000.0, 0.0, 3000.0)]


### Debt Snowball Allocation

In [27]:
# Initialize the allocator.
allocator = SortedAllocator(sort_key='debt_snowball')

# Allocate the balance to the envelopes.
allocator.allocate(envelopes=envelopes, balance=balance, curr_date=start_date)

# Verify total allocated equals budget.
print(sum([e.allocated for e in envelopes if e.allocated is not None]) == balance)
print([(e.bill.bill_id, e.bill.amount_due, e.allocated, e.remaining) for e in envelopes])

True
[('car_insur_1', 774.76, None, None), ('suburu_reg', 191.0, 191.0, 0.0), ('honda_reg', 334.0, 334.0, 0.0), ('prop_tax_supl_2024', 11520.23, 10726.65, 793.5799999999999), ('car_insur', 774.76, 774.76, 0.0), ('prop_tax_1', 13161.15, 0.0, 13161.15), ('prop_tax_2', 13161.15, 0.0, 13161.15), ('home_insur', 3000.0, 3000.0, 0.0)]


### Custom Sorting

In [30]:
# Sort by service type priority
def service_priority(bill: 'BillInstance') -> int:

    priorities = {
        "prop_tax_supl_2024": 1, "prop_tax_1": 2, "prop_tax_2": 3
    }

    # Iterate over the priorities and return the priority of the service
    # type.
    for service, priority in priorities.items():

        # If the service type is in the bill, return the priority.
        if bill.bill_id.lower() == service.lower():
            return priority

    # Default low priority.
    return 99

# Initialize the allocator.
allocator = SortedAllocator(sort_key=service_priority)

# Allocate the balance to the envelopes.
allocator.allocate(envelopes=envelopes, balance=balance, curr_date=start_date)

# Verify total allocated equals budget.
print(sum([e.allocated for e in envelopes if e.allocated is not None]) == balance)
print([(e.bill.bill_id, e.bill.amount_due, e.allocated, e.remaining) for e in envelopes])

True
[('car_insur_1', 774.76, None, None), ('suburu_reg', 191.0, 0.0, 191.0), ('honda_reg', 334.0, 0.0, 334.0), ('prop_tax_supl_2024', 11520.23, 11520.23, 0.0), ('car_insur', 774.76, 0.0, 774.76), ('prop_tax_1', 13161.15, 3506.1800000000003, 9654.97), ('prop_tax_2', 13161.15, 0.0, 13161.15), ('home_insur', 3000.0, 0.0, 3000.0)]
